In [24]:
import numpy as np
import pandas as pd
from pandas import DataFrame, Series

# sklearn stuff
from sklearn.linear_model import SGDClassifier
from sklearn.datasets import make_regression, make_classification
from shrimp.src.validation.model_validation_suite import RegressorValidationSuite, ClassifierValidationSuite
from sklearn.preprocessing import normalize

# error metrics
from sklearn.metrics import roc_auc_score, precision_score, recall_score, accuracy_score, confusion_matrix

In [25]:
X, y = make_classification(10000)
y = np.where(y == 0, -1, y)

In [26]:
# augment X with intercept
X = np.concatenate((np.ones(10000)[:,np.newaxis], X), axis=1)

In [27]:
X.shape

(10000, 21)

### Feature pre-processing

In [39]:
X = normalize(X, axis=1)
# X = X / np.abs(X.max(axis = 0))
# X /= np.sqrt(21)
assert (np.linalg.norm(X, axis=1) <= 1.00000001).all()

### Fit Logistic Regression

In [40]:
# Logistic Regression via SGD
clf = SGDClassifier(loss='log', penalty='l2', l1_ratio=0, alpha=1.5, verbose=True, fit_intercept=False,
                    learning_rate='bolton', eta0=0.4, random_state=2017)

In [41]:
clf.fit(X, y, R=2/3, beta=2.5, gamma=1.5)

-- Epoch 1
Norm: 0.21, NNZs: 21, Bias: 0.000000, T: 10000, Avg. loss: 0.681942, learning_rate: 0.4000
Total training time: 0.00 seconds.
-- Epoch 2
Norm: 0.23, NNZs: 21, Bias: 0.000000, T: 20000, Avg. loss: 0.681753, learning_rate: 0.4000
Total training time: 0.00 seconds.
-- Epoch 3
Norm: 0.20, NNZs: 21, Bias: 0.000000, T: 30000, Avg. loss: 0.681990, learning_rate: 0.3333
Total training time: 0.01 seconds.
-- Epoch 4
Norm: 0.16, NNZs: 21, Bias: 0.000000, T: 40000, Avg. loss: 0.681543, learning_rate: 0.2222
Total training time: 0.01 seconds.
-- Epoch 5
Norm: 0.17, NNZs: 21, Bias: 0.000000, T: 50000, Avg. loss: 0.681651, learning_rate: 0.1667
Total training time: 0.01 seconds.


/Users/andrei/Documents/scikit-learn/sklearn/linear_model/stochastic_gradient.py:131: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  " default tol will be 1e-3." % type(self), FutureWarning)


SGDClassifier(alpha=1.5, average=False, class_weight=None, epsilon=0.1,
       eta0=0.4, fit_intercept=False, l1_ratio=0, learning_rate='bolton',
       loss='log', max_iter=None, n_iter=None, n_jobs=1, penalty='l2',
       power_t=0.5, random_state=2017, shuffle=True, tol=None,
       verbose=True, warm_start=False)

In [42]:
clf.coef_.shape[1]

21

In [43]:
roc_auc_score(y, clf.predict_proba(X)[:,1])

0.83764225878032894

In [44]:
accuracy_score(y, clf.predict(X))

0.75360000000000005

In [45]:
confusion_matrix(y, clf.predict(X), labels = [-1,1])

array([[3515, 1470],
       [ 994, 4021]])

In [13]:
clf.coef_

array([[ 0.04874059, -0.03453356, -0.00367876, -0.04793597,  0.07130633,
         0.01213033,  0.00648209,  0.01978853, -0.01433956, -0.02793671,
        -0.05231319,  0.03778253,  0.01704509, -0.03298822, -0.00594396,
         0.03933785,  0.01110224,  0.0442674 , -0.0104631 ,  0.04087745,
        -0.01632564]])

### BOLT-ON Logistic DP 

#### constant learning rate with strongly-convex loss function 

In [14]:
clf.bolt_on(epsilon = 1, m = 10000)

In [15]:
clf.coef_

array([[ 0.04846948, -0.03202115, -0.00485479, -0.04903067,  0.07065142,
         0.01151456,  0.00596402,  0.01935841, -0.01292029, -0.02918155,
        -0.05266361,  0.03735564,  0.017023  , -0.03271994, -0.00625434,
         0.03989092,  0.01301727,  0.04521685, -0.01138789,  0.04072074,
        -0.01767802]])

In [16]:
roc_auc_score(y, clf.predict_proba(X_unit)[:,1])

0.82880278008104069

In [17]:
accuracy_score(y, clf.predict(X_unit))

0.73640000000000005

In [18]:
confusion_matrix(y, clf.predict(X_unit), labels = [-1,1])

array([[3128, 1893],
       [ 743, 4236]])

In [19]:
# from random import gauss

# def make_rand_vector(dims):
#     vec = np.array([gauss(0, 1) for i in range(dims)])
#     return vec / np.linalg.norm(vec)

# # Parameters for Laplacian noise
# alpha = 1.5
# L = 2
# beta = 1 + alpha
# gamma = alpha
# eta = 0.4

# # num features + intercept
# d= 21

# # L2 sensitivity of the learning algorithm
# L2_sensitivity = (2*eta*L)/(1 - (1-eta*gamma)**X_unit.shape[0])

# # sampling from the associated Laplace distribution
# epsilon = 2
# v = make_rand_vector(d)
# magnitude = np.random.gamma(d, L2_sensitivity/epsilon)
# noise = v*magnitude